<a href="https://colab.research.google.com/github/fernandodelaiglesia/cajondesastre/blob/master/QECwDist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BitFlip QEC with distribution

In [0]:
!pip install projectq

In [0]:
from projectq import MainEngine
from projectq.backends import Simulator
from projectq.ops import StatePreparation, X, Z, H, Measure, All
from projectq.meta import Loop, Control, Compute, Uncompute

import math



# Simple bitflip check

In [0]:
# List of non error and error ancilla result
nonerror_list = [[0, 0, 0], [0, 1, 1], [1, 0, 0], [1, 1, 1]]
error_list = [[0, 0, 1], [0, 1, 0], [1, 0, 1], [1, 1, 0]]

# Quantum process
eng = MainEngine()

info_qbits = eng.allocate_qureg(3)

ancilla1 = eng.allocate_qureg(3)
ancilla2 = eng.allocate_qureg(3)

amplitude0 = math.sqrt(3)/2.
amplitude1 = 1./2.
StatePreparation([amplitude0, amplitude1]) | info_qbits[0]

# Prepare the protected state
# Code: 3 redundant qubit

with Control(eng, info_qbits[0]):
  All(X) | info_qbits[1:]

# Prepare the entagled ancillas

H | ancilla1[0]
with Control(eng, ancilla1[0]):
  All(X) | ancilla1[1:]
  
H | ancilla2[0]
with Control(eng, ancilla2[0]):
  All(X) | ancilla2[1:]

# Distribute
# Nothing to do, this is local simulation

# A: info_qbits[0], ancilla1[1]
# B: info_qbits[1], ancilla1[2], ancilla2[1]
# C: info_qbits[2], ancilla2[2]

# Let's introduce some (bit flip) error

#X | info_qbits[2]

# QEC

# A
with Control(eng, info_qbits[0]):
  X | ancilla1[1]
  
# B
with Control(eng, info_qbits[1]):
  X | ancilla1[2]
  X | ancilla2[1]
  
# C
with Control(eng, info_qbits[2]):
  X | ancilla2[2]
  
# Measure the ancillas to check for errors

H | ancilla1[0]
H | ancilla2[0]

All(Measure) | ancilla1
All(Measure) | ancilla2

m_ancilla1 = [int(q) for q in ancilla1]
m_ancilla2 = [int(q) for q in ancilla2]

print("Ancilla measurement: 1: ",m_ancilla1, " , 2: ",m_ancilla2)

eng.flush()

prob000 = eng.backend.get_probability('000', info_qbits)
prob111 = eng.backend.get_probability('111', info_qbits)

print("Info qubits probability: |000>",prob000, " , |111>: ",prob111, " . It should be 0.75 and 0.25")

# Error detection and error recovery
if (m_ancilla1 in nonerror_list) and (m_ancilla2 in nonerror_list):
  print("\nNo error or evesdrop happens, cool!, No error recovery required\n")
elif (m_ancilla1 in error_list) and (m_ancilla2 in nonerror_list):
  print("\nError or Eve happens :-(  : m_ancilla1 = ", m_ancilla1, ", m_ancilla2 = ", m_ancilla2,"\n")
  X | info_qbits[0]
  print ("Error corrected with X0!\n")
elif (m_ancilla1 in error_list) and (m_ancilla2 in error_list):
  print("\nError or Eve happens :-(  : m_ancilla1 = ", m_ancilla1, ", m_ancilla2 = ", m_ancilla2,"\n")
  X | info_qbits[1]
  print ("Error corrected with X1!\n")
elif (m_ancilla1 in nonerror_list) and (m_ancilla2 in error_list):
  print("\nError or Eve happens :-(  : m_ancilla1 = ", m_ancilla1, ", m_ancilla2 = ", m_ancilla2,"\n")
  X | info_qbits[2]
  print ("Error corrected with X2!\n")


eng.flush()

prob000 = eng.backend.get_probability('000', info_qbits)
prob111 = eng.backend.get_probability('111', info_qbits)

print("Info qubits probability AFTER error recovery: |000>",prob000, " , |111>: ",prob111, " . It should be 0.75 and 0.25")

All(Measure) | info_qbits

eng.flush()



Ancilla measurement: 1:  [0, 1, 1]  , 2:  [0, 1, 1]
Info qubits probability: |000> 0.7500000000002589  , |111>:  0.24999999999974126  . It should be 0.75 and 0.25

No error or evesdrop happens, cool!, No error recovery required

Info qubits probability AFTER error recovery: |000> 0.7500000000002589  , |111>:  0.24999999999974126  . It should be 0.75 and 0.25


# BitFlip check for > 3 qubit
With the check that it can detect more than 3 errors

In [27]:
# List of non error and error ancilla result
nonerror_list = [[0, 0, 0], [0, 1, 1], [1, 0, 0], [1, 1, 1]]
error_list = [[0, 0, 1], [0, 1, 0], [1, 0, 1], [1, 1, 0]]

# Quantum process
eng = MainEngine()

# Number of nodes = number of system qubits

n_nodes = 6
n_ancillas = n_nodes - 1

info_qbits = eng.allocate_qureg(n_nodes)

ancillas = []

for i in range(n_ancillas):
  ancillas.insert(i,eng.allocate_qureg(3))

# The state preparation is very simple, just a half/half (Haddamard)

H | info_qbits[0]

# Prepare the protected state
# Code: 6 redundant qubit

with Control(eng, info_qbits[0]):
  All(X) | info_qbits[1:]

# Prepare the entagled ancillas

for i in range(n_ancillas):
  H | ancillas[i][0]
  with Control(eng, ancillas[i][0]):
    All(X) | ancillas[i][1:]

# Distribute
# Nothing to do, this is local simulation

# A: info_qbits[0], ancilla[0][1]
# B: info_qbits[1], ancilla[0][2], ancilla[1][1]
# C: info_qbits[2], ancilla[1][2]
# etc.

# Let's introduce some (bit flip) error

#X | info_qbits[0]

# QEC (or detection)

for node in range(n_nodes):
  print("Processing node # ", node)
  with Control(eng, info_qbits[node]):
    if node != (n_nodes-1):
      X | ancillas[node][1]
    if node != 0:
      X | ancillas[node-1][2]

# Measure the ancillas to check for errors
print()

m_ancillas = []
for i in range(n_ancillas):
  H | ancillas[i][0]
  All(Measure) | ancillas[i]
  m_ancillas.insert(i, [int(q) for q in ancillas[i]])
  if m_ancillas[i] in nonerror_list:
    print("No parity error in ancilla ", i)
  else:
    print("Parity error in ancilla ", i)

print("\nNo error if ancilla measurement is in:", nonerror_list,)
print("Ancillas measurement: ",m_ancillas)

eng.flush()

All(Measure) | info_qbits

eng.flush()


Processing node #  0
Processing node #  1
Processing node #  2
Processing node #  3
Processing node #  4
Processing node #  5

No parity error in ancilla  0
No parity error in ancilla  1
No parity error in ancilla  2
No parity error in ancilla  3
No parity error in ancilla  4

No error if ancilla measurement is in: [[0, 0, 0], [0, 1, 1], [1, 0, 0], [1, 1, 1]]
Ancillas measurement:  [[0, 1, 1], [0, 1, 1], [1, 0, 0], [0, 1, 1], [1, 0, 0]]


# Bitflip 3 qubit stabilizer

3 qubit stabilizer code:
S = < ZZI, IZZ >, V_s = { |000> , |111>}


In [0]:
# List of non error and error ancilla result
nonerror_list = [[0, 0, 0], [0, 1, 1], [1, 0, 1], [1, 1, 0]]
error_list    = [[0, 0, 1], [0, 1, 0], [1, 0, 0], [1, 1, 1]]

# Quantum process
eng = MainEngine()

info_qbits = eng.allocate_qureg(3)

ancilla1 = eng.allocate_qureg(3)
ancilla2 = eng.allocate_qureg(3)

amplitude0 = math.sqrt(3)/2.
amplitude1 = 1./2.
StatePreparation([amplitude0, amplitude1]) | info_qbits[0]

# Prepare the protected state
# Code: 3 redundant qubit, from the stabilizer <ZZI, IZZ>

with Control(eng, info_qbits[0]):
  All(X) | info_qbits[1:]

# Prepare the entagled ancillas

H | ancilla1[0]
with Control(eng, ancilla1[0]):
  All(X) | ancilla1[1:]
  
H | ancilla2[0]
with Control(eng, ancilla2[0]):
  All(X) | ancilla2[1:]

# Distribute
# Nothing to do, this is local simulation

# A: info_qbits[0], ancilla1[1]
# B: info_qbits[1], ancilla1[2], ancilla2[1]
# C: info_qbits[2], ancilla2[2]

# Let's introduce some (bit flip) error

#X | info_qbits[1]

eng.flush()

prob000 = eng.backend.get_probability('000', info_qbits)
prob111 = eng.backend.get_probability('111', info_qbits)

print("Info qubits probability: |000>",prob000, " , |111>: ",prob111, " . It should be 0.75 and 0.25")

# QEC

# A
with Control(eng, ancilla1[1]):
  Z | info_qbits[0]
  
# B
with Control(eng, ancilla1[2]):
  Z | info_qbits[1]
with Control(eng, ancilla2[1]):
  Z | info_qbits[1]
  
# C
with Control(eng, ancilla2[2]):
  Z | info_qbits[2]

# Measure the ancillas to check for errors.
# Required to H all the ancillas previous to the measure.
# The H and the measure is presented separatelly just to represents
#       the space separation

# No dependency on the location (can be executed where the ancillas
#       are created)
H | ancilla1[0]
Measure | ancilla1[0]
H | ancilla2[0]
Measure | ancilla2[0]

# A
H | ancilla1[1]
Measure | ancilla1[1]

# B
H | ancilla1[2]
H | ancilla2[1]
Measure | ancilla1[2]
Measure | ancilla2[1]

# C
H | ancilla2[2]
Measure | ancilla2[2]

m_ancilla1 = [int(q) for q in ancilla1]
m_ancilla2 = [int(q) for q in ancilla2]

print("Ancilla measurement: 1: ",m_ancilla1, " , 2: ",m_ancilla2)

eng.flush()

prob000 = eng.backend.get_probability('000', info_qbits)
prob111 = eng.backend.get_probability('111', info_qbits)

print("Info qubits probability: |000>",prob000, " , |111>: ",prob111, " . It should be 0.75 and 0.25")

# Error detection and error recovery
if (m_ancilla1 in nonerror_list) and (m_ancilla2 in nonerror_list):
  print("\nNo error or evesdrop happens, cool!, No error recovery required\n")
elif (m_ancilla1 in error_list) and (m_ancilla2 in nonerror_list):
  print("\nError or Eve happens :-(  : m_ancilla1 = ", m_ancilla1, ", m_ancilla2 = ", m_ancilla2,"\n")
  X | info_qbits[0]
  print ("Error corrected with X0!\n")
elif (m_ancilla1 in error_list) and (m_ancilla2 in error_list):
  print("\nError or Eve happens :-(  : m_ancilla1 = ", m_ancilla1, ", m_ancilla2 = ", m_ancilla2,"\n")
  X | info_qbits[1]
  print ("Error corrected with X1!\n")
elif (m_ancilla1 in nonerror_list) and (m_ancilla2 in error_list):
  print("\nError or Eve happens :-(  : m_ancilla1 = ", m_ancilla1, ", m_ancilla2 = ", m_ancilla2,"\n")
  X | info_qbits[2]
  print ("Error corrected with X2!\n")


eng.flush()

prob000 = eng.backend.get_probability('000', info_qbits)
prob111 = eng.backend.get_probability('111', info_qbits)

print("Info qubits probability AFTER error recovery: |000>",prob000, " , |111>: ",prob111, " . It should be 0.75 and 0.25")

All(Measure) | info_qbits

eng.flush()


Info qubits probability: |000> 0.0  , |111>:  0.0  . It should be 0.75 and 0.25
Ancilla measurement: 1:  [0, 1, 0]  , 2:  [0, 0, 1]
Info qubits probability: |000> 0.0  , |111>:  0.0  . It should be 0.75 and 0.25

Error or Eve happens :-(  : m_ancilla1 =  [0, 1, 0] , m_ancilla2 =  [0, 0, 1] 

Error corrected with X1!

Info qubits probability AFTER error recovery: |000> 0.7500000000002589  , |111>:  0.24999999999974137  . It should be 0.75 and 0.25
